In [17]:
import pandas as pd
import json
import os
from tqdm import tqdm

with open('accept-period.json') as fopen:
    period = set(json.load(fopen))
len(period)

977091

In [4]:
df = pd.read_parquet('chunk-streaming-flatten.parquet')

In [6]:
processed = pd.read_parquet('processed.parquet')

In [8]:
processed.iloc[0]

reference_text              Uhm, hello, selamat pagi ye, saya dari custome...
generate_text               Encik, bolehkah Encik memberikan maklum balas ...
normalized_generate_text    Encik, bolehkah Encik memberikan maklum balas ...
reference_audio                                          husein-assistant.mp3
filename_audio                                   response-husein-v3/55420.mp3
speaker                                                                husein
similarity                                                           0.800952
audio_length                                                         4.400181
index                                                                 1367694
alignment                   [{'end': 0.38, 'score': -3.89, 'start': 0.12, ...
averaged_pitch              [279.365, 104.309, 97.318, 94.798, 95.233, 91....
distances                   [0.073, 0.007, 0.012, 0.006, 0.007, 0.012, 0.0...
Name: 0, dtype: object

In [16]:
filenames = set()
for i in tqdm(range(len(processed))):
    if i not in period:
        continue
    f = processed.iloc[i]['filename_audio'].replace('/', '_').replace('.mp3', '')
    filenames.add(f)
len(filenames)

100%|██████████| 1645455/1645455 [00:28<00:00, 57632.93it/s]


977091

In [44]:
import re

def clean(string):
    string = re.sub('[^A-Za-z ]+', ' ', string)
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string.lower()

accepted = []
already = set()
for i in tqdm(range(len(df))):
    splitted = os.path.split(df.iloc[i]['filename_audio'])[1].split('_')[:-1]
    splitted = '_'.join(splitted)
    speaker = 'husein' if 'husein' in df.iloc[i]['filename_audio'] else 'idayu'
    if splitted not in filenames:
        continue
    t = speaker + clean(df.iloc[i]['text'])
    if t in already:
        continue

    accepted.append(i)
    already.add(t)

100%|██████████| 5327569/5327569 [03:20<00:00, 26548.38it/s]


In [45]:
len(accepted)

2014337

In [49]:
with open('accept-chunk-streaming-flatten.json', 'w') as fopen:
    json.dump(list(accepted), fopen)